# Cleaning the data

I will start by reading the data from all_data.csv which contains the training set and test set joined together.

In [1]:
import settings
import os

import pandas as pd

data = pd.read_csv(os.path.join(settings.PROCESSED_DATA_DIR, "all_data.csv"), index_col =0)

In [2]:
print data.shape
data.head()

(74250, 40)


,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,subset
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,training
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,training
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,training
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,training
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,training


# Feature Selection

Based on their names, some features may capture similar information, so let's explore the data and elimniate the redundunt columns.

In [3]:
columns_to_keep = list(data.columns)
print columns_to_keep

['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'subset']


## scheme_management , scheme_name

In [4]:
data.groupby(['scheme_management', 'scheme_name'])['scheme_name'].count()

scheme_management  scheme_name                 
Company            BFFS                              1
                   BSF water supplying               1
                   Bagamoyo Wate                    76
                   Bagamoyo wate                   272
                   Borehole                          1
                   Borehole drilling project         1
                   Bulamba piped scheme              3
                   Burieni water supply              1
                   Doe water supply                  4
                   EKTM 2 water  supply              1
                   EKTM 2 water suply               28
                   EKTM 2 water supply              10
                   EKTM 3 water supply              59
                   Fufulamsuri water supply          1
                   GRUMETI                          10
                   HESAWA                            1
                   Hakwe water supply               12
                 

I will remove the 'scheme_name' as it has too many unique values.

In [5]:
columns_to_keep.remove('scheme_name')

## extraction_type, extraction_type_group and extraction_type_class

In [6]:
data.groupby([ 'extraction_type_class', 'extraction_type_group','extraction_type'])['extraction_type'].count()

extraction_type_class  extraction_type_group  extraction_type          
gravity                gravity                gravity                      33263
handpump               afridev                afridev                       2208
                       india mark ii          india mark ii                 3029
                       india mark iii         india mark iii                 135
                       nira/tanira            nira/tanira                  10205
                       other handpump         other - mkulima/shinyanga        2
                                              other - play pump              101
                                              other - swn 81                 284
                                              walimi                          60
                       swn 80                 swn 80                        4588
motorpump              mono                   mono                          3628
                       other motorpum

** I will only keep the extraction_type column and remove the two others. I will also combine some levels together based on brand :**
+ india mark ii and india mark iii ==> india mark
+ other - swn 81 and swn 80 ==> swn
+ other - mkulima/shinyanga , other - play pump and walimi ==> other handpump
+ climax and cemo ==> other motorpump

In [7]:
data['extraction_type'].loc[data['extraction_type'] == 'india mark ii'] = 'india mark'
data['extraction_type'].loc[data['extraction_type'] == 'india mark iii'] = 'india mark'
data['extraction_type'].loc[data['extraction_type'] == 'other - swn 81'] = 'swn'
data['extraction_type'].loc[data['extraction_type'] == 'swn 80'] = 'swn'
data['extraction_type'].loc[data['extraction_type'] == 'other - mkulima/shinyanga'] = 'other handpump'
data['extraction_type'].loc[data['extraction_type'] == 'other - play pump'] = 'other handpump'
data['extraction_type'].loc[data['extraction_type'] == 'walimi'] = 'other handpump'
data['extraction_type'].loc[data['extraction_type'] == 'climax'] = 'other motorpump'
data['extraction_type'].loc[data['extraction_type'] == 'cemo'] = 'other motorpump'

columns_to_keep.remove('extraction_type_class')
columns_to_keep.remove('extraction_type_group')

C:\Users\ra18m\Anaconda\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
data.groupby([ 'extraction_type_class', 'extraction_type_group','extraction_type'])['extraction_type'].count()

extraction_type_class  extraction_type_group  extraction_type  
gravity                gravity                gravity              33263
handpump               afridev                afridev               2208
                       india mark ii          india mark            3029
                       india mark iii         india mark             135
                       nira/tanira            nira/tanira          10205
                       other handpump         other handpump         163
                                              swn                    284
                       swn 80                 swn                   4588
motorpump              mono                   mono                  3628
                       other motorpump        other motorpump        149
other                  other                  other                 8102
rope pump              rope pump              other - rope pump      572
submersible            submersible            ksb           

## management_group and management

In [9]:
data.groupby(['management_group','management'])['management'].count()

management_group  management      
commercial        company               859
                  private operator     2504
                  trust                 105
                  water authority      1123
other             other                1083
                  other - school        126
parastatal        parastatal           2229
unknown           unknown               683
user-group        vwc                 50624
                  water board          3688
                  wua                  3118
                  wug                  8108
Name: management, dtype: int64

** I will only keep the "management" column. **

In [10]:
columns_to_keep.remove('management_group')

## payment and payment_type

In [11]:
data.groupby(['payment_type','payment'])['payment'].count()

payment_type  payment              
annually      pay annually              4570
monthly       pay monthly              10397
never pay     never pay                31712
on failure    pay when scheme fails     4842
other         other                     1314
per bucket    pay per bucket           11266
unknown       unknown                  10149
Name: payment, dtype: int64

** The two column "payment" and "payment_type" are similar, i choose to keep the "payment_type" column. **

In [12]:
columns_to_keep.remove('payment')

## water_quality and quality_group

In [13]:
data.groupby(['quality_group','water_quality'])['water_quality'].count()

quality_group  water_quality     
colored        coloured                623
fluoride       fluoride                244
               fluoride abandoned       23
good           soft                  63505
milky          milky                  1005
salty          salty                  6082
               salty abandoned         423
unknown        unknown                2345
Name: water_quality, dtype: int64

** I will keep the "water_quality" column as it is more precise. **

In [14]:
columns_to_keep.remove('quality_group')

## quantity and quantity_group

In [15]:
data.groupby(['quantity', 'quantity_group'])['quantity'].count()

quantity      quantity_group
dry           dry                7782
enough        enough            41522
insufficient  insufficient      18896
seasonal      seasonal           5075
unknown       unknown             975
Name: quantity, dtype: int64

** This two columns are similar, i choose to keep the "quantity" column. **

In [16]:
columns_to_keep.remove('quantity_group')

## source, source_type and source_class

In [17]:
data.groupby(['source_class', 'source_type', 'source'])['source'].count()

source_class  source_type           source              
groundwater   borehole              hand dtw                 1108
                                    machine dbh             13822
              shallow well          shallow well            21140
              spring                spring                  21216
surface       dam                   dam                       840
              rainwater harvesting  rainwater harvesting     2863
              river/lake            lake                      950
                                    river                   11964
unknown       other                 other                     261
                                    unknown                    86
Name: source, dtype: int64

** I will only keep the "source" column as it carries more specific information.I will also set the unknown values to other. **

In [18]:
columns_to_keep.remove('source_class')
columns_to_keep.remove('source_type')
data['source'].loc[data['source'] == 'unknown'] = 'other'

In [19]:
print columns_to_keep

['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'construction_year', 'extraction_type', 'management', 'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type', 'waterpoint_type_group', 'subset']


## waterpoint_type and waterpoint_type_group

In [20]:
data.groupby(['waterpoint_type_group', 'waterpoint_type'])['waterpoint_type'].count()

waterpoint_type_group  waterpoint_type            
cattle trough          cattle trough                    150
communal standpipe     communal standpipe             35628
                       communal standpipe multiple     7611
dam                    dam                                8
hand pump              hand pump                      21884
improved spring        improved spring                  959
other                  other                           8010
Name: waterpoint_type, dtype: int64

** I will remove the "waterpoint_type_group" column. ** 

In [21]:
columns_to_keep.remove('waterpoint_type_group')

## Geographic information

We have many features that describe the location :
+ region, (region_code)
+ district_code
+ subvillage
+ lga
+ ward
+ longitude, latitude

I will keep the region_code and the ward as categorical feature and longitude x latitude as numerical feature

In [22]:
columns_to_keep.remove('region')
columns_to_keep.remove('district_code')
columns_to_keep.remove('subvillage')
#columns_to_keep.remove('ward')

** It seems that the 'lga' column captures some interesting information. For some waterpoints, It specifies the nature of the area (rural or urban). So based on the lga column, I will add a new feature called "area" that can be either rural, urban or other. **

In [23]:
def build_area_column(lga):
    if "rural" in lga.lower():
        return "rural"
    elif "urban" in lga.lower():
        return "urban"
    else :
        return "other"

data["area"] = data['lga'].map(build_area_column)
columns_to_keep.append("area")

In [24]:
data['area'].value_counts()

other    60318
rural    11814
urban     2118
Name: area, dtype: int64

Finally I remove the lga column.

In [25]:
columns_to_keep.remove("lga")

In [26]:
columns_to_keep

['amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'region_code',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'permit',
 'construction_year',
 'extraction_type',
 'management',
 'payment_type',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type',
 'subset',
 'area']

## Date_recorded

Whether a pump do function or do not may also depend on seasons. According to [safaribookings](https://www.safaribookings.com/tanzania/climate) *Tanzania has two seasons : A Dry season from June to October and a Wet season from November to May.*
So in order to capture the seasonal effect, i will convert the "date_recorded" feature into two columns :
+ "year_recorded" : indicates the year in which the waterpump was recorded.
+ "month_recorded" : indicates the month in which the waterpump was recorded (from 1 to 12).

In [27]:
data['year_recorded'] = data['date_recorded'].map(lambda x: pd.to_datetime(x).year)
data['day_recorded'] = data['date_recorded'].map(lambda x: pd.to_datetime(x).dayofyear)

In [28]:
columns_to_keep.append('year_recorded')
columns_to_keep.append('day_recorded')
# I will remove the 'date_recorded' column
columns_to_keep.remove('date_recorded')

I will also remove the 'num_private' and 'wpt_name' variables as they do not have any effect on whether a waterpump operate or not. 

In [29]:
columns_to_keep.remove('wpt_name')
columns_to_keep.remove('num_private')

Handling missing values
====


Let's see which column have missing values.

In [30]:
data[columns_to_keep].isnull().sum()

amount_tsh              0
funder               4504
gps_height              0
installer            4532
longitude               0
latitude                0
basin                   0
region_code             0
ward                    0
population              0
public_meeting       4155
recorded_by             0
scheme_management    4846
permit               3793
construction_year       0
extraction_type         0
management              0
payment_type            0
water_quality           0
quantity                0
source                  0
waterpoint_type         0
subset                  0
area                    0
year_recorded           0
day_recorded            0
dtype: int64

** I will set the missing values in the "funder", "installer", "scheme_management" columns to 'Others'. **

In [31]:
data['funder'] = data['funder'].fillna('Others')
data['installer'] = data['installer'].fillna('Others')

In [32]:
data['scheme_management'].value_counts()

VWC                 45917
WUG                  6496
Water authority      3975
WUA                  3551
Water Board          3462
Parastatal           2124
Company              1341
Private operator     1326
Other                 996
SWC                   123
Trust                  92
None                    1
Name: scheme_management, dtype: int64

In [33]:
data['scheme_management'] = data['scheme_management'].fillna('Other')
data[data['scheme_management'] == 'None' ] = 'Other'

Let's now explore the "public_meeting" and "permit" columns

In [34]:
print data['public_meeting'].value_counts()
print data['permit'].value_counts()

True     63748
False     6346
Other        1
Name: public_meeting, dtype: int64
True     48606
False    21850
Other        1
Name: permit, dtype: int64


I will set the missing values to the most frequent value in both columns, which is "True".

In [35]:
data['public_meeting'] = data['public_meeting'].fillna(True)
data['permit'] = data['permit'].fillna(True)

### Longitude and Latitude

It looks like many rows have abnormal values in their longitude and latitude attributes which can be considered as missing values.

In [36]:
data[['longitude','latitude']].loc[(data["longitude"]==0) | (data["latitude"]==0)]

,longitude,latitude
id,,
6091,0,-2e-08
32376,0,-2e-08
72678,0,-2e-08
56725,0,-2e-08
13042,0,-2e-08
33500,0,-2e-08
71487,0,-2e-08
70379,0,-2e-08
59713,0,-2e-08


Since the district_code column has no missing values, we can use it to approximate the lat/long missing values.

First let's create a new column which contains the region_code and district_code joined together

In [37]:
data['region_and_district'] = data['region_code'].map(str) + "," + data['district_code'].map(str)

Then I define a function that takes the region_code and the district_code as input and return the latitude mean of the district:

In [38]:
def lat_approx(region):
    return data['latitude'].loc[(data["longitude"]!=0) & (data["latitude"]!=0) & (data['region_code'] == region)].mean()

Same for the longitude:

In [39]:
def long_approx(region):
    return data['longitude'].loc[(data["longitude"]!=0) & (data["latitude"]!=0) & (data['region_code'] == region)].mean()

Finally, i apply those two functions on the latitude and longitude columns to set any missing value to the mean latitude (and longitude) of the corresponding district.

In [40]:
data['latitude'].loc[(data["longitude"]==0) | (data["latitude"]==0)] = data['region_code'].loc[(data["longitude"]==0) | (data["latitude"]==0)].map(lat_approx)
data['longitude'].loc[(data["longitude"]==0) | (data["latitude"]==0)] = data['region_code'].loc[(data["longitude"]==0) | (data["latitude"]==0)].map(long_approx)

I will remove the "amount_tsh" column as it has too many missing values. (I consider the value of 0 in the amount_tsh variable as missing value because it does not make sense)

In [41]:
data[data['amount_tsh'] == 0]['amount_tsh'].count()/float(data.shape[0])

0.70099663299663295

==> 70 % of the amount_tsh values are missing

In [42]:
columns_to_keep.remove('amount_tsh')

### 'Recorded_by' column

In [43]:
data['recorded_by'].value_counts()

GeoData Consultants Ltd    74249
Other                          1
Name: recorded_by, dtype: int64

All the data was recorded by 'GeoData Consultants Ltd', so I delete this column.

In [44]:
columns_to_keep.remove('recorded_by')

I found that there is a row that has the value of 'Other' in all of its attributes, so let's drop this row.

In [45]:
data.loc[data['longitude'] == 'Other']

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,source,source_type,source_class,waterpoint_type,waterpoint_type_group,subset,area,year_recorded,day_recorded,region_and_district
id,,,,,,,,,,,,,,,,,,,,,
23849,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,...,Other,Other,Other,Other,Other,Other,Other,Other,Other,"Other,Other"


In [46]:
data = data.drop(23849)

Now let's list the columns that I will keep for making predictions

In [47]:
print columns_to_keep

['funder', 'gps_height', 'installer', 'longitude', 'latitude', 'basin', 'region_code', 'ward', 'population', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type', 'management', 'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type', 'subset', 'area', 'year_recorded', 'day_recorded']


In [48]:
data_cleaned = data[columns_to_keep]
data_cleaned.head()

,funder,gps_height,installer,longitude,latitude,basin,region_code,ward,population,public_meeting,...,management,payment_type,water_quality,quantity,source,waterpoint_type,subset,area,year_recorded,day_recorded
id,,,,,,,,,,,,,,,,,,,,,
69572,Roman,1390,Roman,34.9381,-9.85632,Lake Nyasa,11,Mundindi,109,True,...,vwc,annually,soft,enough,spring,communal standpipe,training,other,2011,73
8776,Grumeti,1399,GRUMETI,34.6988,-2.14747,Lake Victoria,20,Natta,280,True,...,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,training,other,2013,65
34310,Lottery Club,686,World vision,37.4607,-3.82133,Pangani,21,Ngorika,250,True,...,vwc,per bucket,soft,enough,dam,communal standpipe multiple,training,other,2013,56
67743,Unicef,263,UNICEF,38.4862,-11.1553,Ruvuma / Southern Coast,90,Nanyumbu,58,True,...,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,training,other,2013,28
19728,Action In A,0,Artisan,31.1308,-1.82536,Lake Victoria,18,Nyakasimbi,0,True,...,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,training,other,2011,194


In [49]:
data_cleaned.to_csv('processed/all_data_cleaned.csv', encoding='utf-8')